# mBert
 = Multilingual Bidirectional Encoder Representations from Transformers

 mBERT ist ein mehrsprachiges Modell, das Texte in verschiedenen Sprachen verarbeiten kann, ohne sprachspezifische Modelle zu benötigen. Es misst die semantische Ähnlichkeit von Texten anhand von Embeddings, die Wörter in numerische Vektoren umwandeln. Diese Vektoren erfassen die Bedeutung der Wörter und ihre Beziehungen zueinander. Der Similarity Score basiert auf der Ähnlichkeit der Vektoren, unabhängig von der Sprache der Texte. So kann mBERT die semantische Ähnlichkeit zwischen Texten in verschiedenen Sprachen messen.

### Vorteile:

- Bidirektionales Encoding: Transformerencoder, Berücksichtigung des Kontext vor und nach einem Wort (Umfassendes Textverständnis)

- Multilinguales Training: Lernen syntaktischer Strukturen, Semantik, Kontext auf verschiedenen Sprachen

- Transferlernen: Modell ist für viele Sprachen geeignet, wodurch nicht für jede Sprache ein seperates Modell benötigt wird

- Sprachverständnis auf Wortebene: Versteht Kontext jedes Wortes besser

> Die Messung der Ergebnisse erfolgt durch den Similarity Score. Je höher dieser ist, desto ähnlicher sind sich die Texte.

Quellen: 
- https://arxiv.org/pdf/1907.11692
- https://github.com/huggingface/transformers

In [1]:
# Imports
import os
import json
from transformers import BertTokenizer, BertModel
import torch
from torch.nn.functional import cosine_similarity

c:\Users\sandra.nuissl\Desktop\EvaluationOfTranslation\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Laden des Modells und des Tokenizers (Es könnten hier auch andere Modelle herangezogen werden, je nach Fokus)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

In [3]:
# Pfad der JSONS ermitteln
folder = "../Data/atticus"
list_files = os.listdir(folder)

# Leere Liste für englische und deutsche Texte
list_english_text = []
list_german_text = []

# Laden der Files und Extraktion der englischen Texte (Source Language = en-US)
for path in list_files:
    path_complete = folder + "//" + path
    with open(path_complete,'r', encoding="utf-8-sig") as file:
        obj = json.load(file)

    # Iterieren durch die Texte der JSON und Selection der englischen Texte
    for text in obj:
        # Selection der englischen Sätze
        list_english_text.append(text['Text'])
        # Überprüfen, ob die Sprache "de-DE" ist und Speichern in einer Liste
        for translation in text['Translations']:
            if translation['Language'] == 'de-DE':
                list_german_text.append(translation['Text'])

In [4]:
# Erstellen der Datensätze
source_sentence = list_english_text[20]
translated_sentence = list_german_text[20]

In [5]:
# Tokenize Sätze
source_inputs = tokenizer(source_sentence, return_tensors='pt')
translated_inputs = tokenizer(translated_sentence, return_tensors='pt')

In [6]:
# Daten embedding
with torch.no_grad():
    source_outputs = model(**source_inputs)
    translated_outputs = model(**translated_inputs)

# Anwendung des Embeddings
source_embedding = source_outputs.last_hidden_state.mean(dim=1)
translated_embedding = translated_outputs.last_hidden_state.mean(dim=1)

In [7]:
# Similarity Score kalkulieren
similarity = cosine_similarity(source_embedding, translated_embedding)
similarity_score = similarity.item()

# Ausgabe
print(f"Similarity score: {similarity_score}")

Similarity score: 0.5728966593742371
